In [7]:
# 정의부분 -> 그냥 실행해 주세요
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from matplotlib import pyplot as plt
import pandas as pd

## confusion_matrix 출력
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    # 사이즈 변경: figsize = (x, y) -> x, y로 사이즈를 변경해 주세요
    fig, ax = plt.subplots(figsize=(30, 30)) 
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.show()

In [8]:
# 오차분석을 하고 싶은 csv파일 경로를 지정해 주세요
# 기본으로 예측결과는, 'code/prediction/{inference_file}/{val_inference.csv}' 에 있습니다
pred_path = './code/prediction/2023-05-12-12:06:44/val_inference.csv' # 지정요
pred_df = pd.read_csv(pred_path)
# pred_df.columns

Index(['id', 'sentence', 'subject_entity', 'object_entity', 'label',
       'pred_label', 'probs'],
      dtype='object')

In [10]:
labels = pred_df['label']
pred_label = pred_df['pred_label']
# plot_confusion_matrix(pred_label, labels, labels.unique())